# PDBe API Training

### PDBe search

This interactive Python notebook will guide you through programmatically accessing Protein Data Bank in Europe (PDBe)
data using our REST API

The REST API is a programmatic way to obtain information from the PDB and EMDB. You can access details about:

* sample
* experiment
* models
* compounds
* cross-references
* publications
* quality
* assemblies
* search
and more...
For more information, visit https://pdbe.org/api


This tutorial will guide you through searching PDBe programmatically.


First we will import the code which will do the work

In [1]:
from python_modules.api_modules import run_search

Now we are ready to actually run a search against the PDB API for entries containing human Dihydrofolate reductase in the PDB. This will return a list of results - only 10 to start with.

A list of search terms is available at:
https://www.ebi.ac.uk/pdbe/api/doc/search.html

This will return details of human Dihydrofolate reductase's in the PDB

The search terms are defined as a dictionary (a hash in other programming lanuguages).
e.g.
{"molecule_name":"Dihydrofolate reductase"}
Here we are searching for molecules named Dihydrofolate reductase.
If we search for two terms i.e. molecule_name and organism_scientific_name then we will get molecules that match both search terms.

We will return the number of results for two searches.

The first one will hit the limit of 100. There are more than 100 Dihydrofolate reductase structures.
We have to add the argument "number_of_rows" to a higher number, say 1000, to find all the examples.

In [2]:
print('1st search')
search_terms = {"molecule_name":"Dihydrofolate reductase"}

first_results = run_search(search_terms)
number_of_results = len(first_results)
print('Number of results')
print(number_of_results)
print('finished')

1st search
https://www.ebi.ac.uk/pdbe/search/pdb/select?q=molecule_name:"Dihydrofolate reductase"&wt=json&rows=100
Number of results for Dihydrofolate reductase: 100
Number of results
100
finished


We will then increase the number of rows to 1000

In [ ]:
print('Search with 1000 rows')
first_results_more_rows = run_search(search_terms, number_of_rows=1000)
print('Number of results: {}'.format(len(first_results_more_rows)))
print('finished')